---



# Unit 2 - Part 3a: Chain of Thought (CoT)

## 1. Introduction: The Inner Monologue

Standard LLMs try to jump straight to the answer. For complex problems (math, logic), this often fails.

**Chain of Thought (CoT)** forces the model to "think out loud" before answering.

### Why use a "Dumb" Model?
For this unit, we will use **Llama3.1-8b** (via Groq). It is a smaller, faster model.
Why? Because huge models (like Gemini Pro or GPT-4) are often *too smart*—they solve logic riddles instantly without thinking.

To really see the power of Prompt Engineering, we need a model that **needs help**.

### Visualizing the Process (Flowchart)
```mermaid
graph TD
    Input[Question: 5+5*2?]
    Input -->|Standard| Wrong[Answer: 20 (Wrong)]
    Input -->|CoT| Step1[Step 1: 5*2=10]
    Step1 --> Step2[Step 2: 5+10=15]
    Step2 --> Correct[Answer: 15 (Correct)]
```

## 2. Concept: Latent Reasoning

Why does this work?
Because LLMs are "Next Token Predictors".
- If you force it to answer immediately, it must predict the digits `1` and `5` immediately.
- If you let it "think", it generates intermediate tokens (`5`, `*`, `2`, `=`, `1`, `0`).
- The model then **ATTENDS** to these new tokens to compute the final answer.

**Writing is Thinking.**

In [2]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

Enter your Groq API Key: ··········


## 3. The Experiment: A Tricky Math Problem

Let's try a problem that requires multi-step logic.

**Problem:**
"Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

In [3]:
question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (5) to the number of tennis balls he bought (6):

5 + 6 = 11

So, Roger now has 11 tennis balls.


### Critique
Smaller models often latch onto the visible numbers (5 and 2) and simply add them (7), ignoring the multiplication step implied by "cans".

Let's force it to think.

In [4]:
# 2. CoT Prompt (Magic Phrase)
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger already has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 more tennis balls.
3. Now, we add the tennis balls he already had (5) to the tennis balls he bought (6). 5 + 6 = 11

So, Roger now has 11 tennis balls.


## 4. Analysis

Look at the output. By explicitly breaking it down:
1.  "Roger starts with 5."
2.  "2 cans * 3 balls = 6 balls."
3.  "5 + 6 = 11."

The model effectively "debugs" its own logic by generating the intermediate steps.

---



# Unit 2 - Part 3b: Tree of Thoughts (ToT) & Graph of Thoughts (GoT)

## 1. Introduction: Beyond A -> B

CoT is linear. But complex reasoning is often nonlinear. We need to explore branches (ToT) or even combine ideas (GoT).

We continue using **Llama3.1-8b via Groq** to show how structure improves performance.

In [5]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed

## 2. Tree of Thoughts (ToT)

ToT explores multiple branches before making a decision.
**Analogy:** A chess player considering 3 possible moves before playing one.

### Implementation
We will generate 3 distinct solutions for a problem and then use a "Judge" to pick the best one.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 5-year-old to eat vegetables?"

# Step 1: The Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: The Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Child Psychologist. Pick the most sustainable one (not bribery) and explain why.
    """
)

# Chain: Input -> Branches -> Judge -> Output
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
As a Child Psychologist, I would recommend **Solution 1: Create a "Veggie Face" on Their Plate** as the most sustainable approach. Here's why:

1. **Non-bribery**: Unlike some other approaches that may involve hidden veggies or rewards, this solution encourages healthy eating habits without resorting to bribery or deception.
2. **Involvement and Engagement**: By letting your child help create the "veggie face," you're engaging them in the process and making mealtime a collaborative experience. This can help build their confidence and sense of ownership.
3. **Exploration and Discovery**: The creative presentation of vegetables can spark curiosity and encourage your child to explore the different flavors, textures, and colors of various veggies.
4. **Development of Fine Motor Skills**: Cutting and arranging vegetables can help develop your child's fine motor skills, hand-eye coordination, and creativity.
5. **Positive Associations**: By making mealti

## 3. Graph of Thoughts (GoT)

You asked: **"Where is Graph of Thoughts?"**

GoT is more complex. It's a network. Information can split, process specific parts, and then **AGGREGATE** back together.

### The Workflow (Writer's Room)
1.  **Split:** Generate 3 independent story plots (Sci-Fi, Fantasy, Mystery).
2.  **Aggregate:** The model reads all 3 and creates a "Master Plot" that combines the best elements of each.
3.  **Refine:** Polish the Master Plot.

```mermaid
graph LR
   Start(Concept) --> A[Draft 1]
   Start --> B[Draft 2]
   Start --> C[Draft 3]
   A & B & C --> Mixer[Aggregator]
   Mixer --> Final[Final Story]
```

In [7]:
# 1. The Generator (Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

# 2. The Aggregator (Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

# 3. The Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
In "Echoes of Eternity," a brilliant physicist, Emma, discovers a way to manipulate time, but her groundbreaking invention comes with a terrifying cost. As she navigates her complicated past and a forbidden love with a mysterious stranger, Jack, she realizes that her actions in the past are creating a series of alternate realities. However, each time she travels through time, a dark entity from the 19th century, known only as "The Devourer," follows, feeding on the fear and chaos of each new reality. As Emma's love for Jack deepens, and the stakes grow higher, she must confront the darkness that threatens to consume her, all while racing against time to prevent a catastrophic future and protect the very fabric of her existence.


## 4. Summary & Comparison Table

| Method | Structure | Best For... | Cost/Latency |
|--------|-----------|-------------|--------------|
| **Simple Prompt** | Input -> Output | Simple facts, summaries | ⭐ Low |
| **CoT (Chain)** | Input -> Steps -> Output | Math, Logic, Debugging | ⭐⭐ Med |
| **ToT (Tree)** | Input -> 3x Branches -> Select -> Output | Strategic decisions, Brainstorming | ⭐⭐⭐ High |
| **GoT (Graph)** | Input -> Branch -> Mix/Aggregate -> Output | Creative Writing, Research Synthesis | ⭐⭐⭐⭐ V. High |

**Recommendation:** Start with CoT. Only use ToT/GoT if CoT fails.